In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import pandas as pd
from elo import *

### Load all data

In [83]:
score_features = pd.read_pickle('score_features.pickle')
game_features = pd.read_pickle('game_features.pickle')
ecos = get_ecos()
ecos = pd.get_dummies(ecos)
game_results = pd.read_pickle('results.pickle')
game_results = pd.get_dummies(game_results[::2]).reset_index()
game_features.shape, score_features.shape, game_results.shape, ecos.shape

((50000, 24), (50000, 14), (50000, 4), (50000, 495))

In [85]:
X = pd.concat([game_features, score_features, game_results, ecos], axis=1, ignore_index=True)
X.to_pickle('all_features.pickle')

In [86]:
X

,0,1,2,3,4,5,6,7,8,9,...,527,528,529,530,531,532,533,534,535,536
0,5,0,0,1,1,1,6,1,2,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2,1,2,7,1,2,2,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,1,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,7,0,0,0,1,1,4,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,2,1,1,4,1,2,0,...,0,0,0,0,0,0,0,0,0,0
5,6,0,0,0,1,0,5,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,1,1,0,4,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,14,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,1,0,5,0,1,1,...,0,0,0,0,0,0,0,0,0,0
9,0,14,0,1,1,0,5,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### Score features

In [25]:
import numpy as np

stockfish = pd.read_csv('stockfish.csv')
stockfish.MoveScores = stockfish.MoveScores.str.replace('NA','').str.split()
stockfish.MoveScores = stockfish.MoveScores.apply(lambda x: np.asarray(x, dtype=int))
stockfish.drop('Event', axis=1, inplace=True)
stockfish.index = list(range(0, 2*len(ecos),2))
stockfish.head()

,MoveScores
0,"[18, 17, 12, 8, -5, 12, 3, -2, 22, 21, 20, 13,..."
2,"[26, 44, 26, 18, 14, 34, 36, 31, 37, 35, 42, 5..."
4,"[26, 51, 68, 57, 65, 77, 48, 93, 61, 63, 63, 5..."
6,"[2, 21, 5, 53, 35, 45, 37, 54, 10, 22, 8, 48, ..."
8,"[26, 64, 35, 53, 18, 20, 18, 20, 10, 49, 60, 9..."


In [44]:
features = defaultdict(float)
row = 0
scores = np.r_[0, stockfish.ix[row, 0]]
white_scores = scores[::2]
black_scores = scores[1::2]
diffs = np.diff(scores)
white_diffs = diffs[::2]
black_diffs = diffs[1::2]
features['mean_abs_error'] = np.abs(diffs).sum() / len(scores)
white_diffs

array([ 18,  -5, -13,  -9,  24,  -1,  -5, -10,  -9,  -4, -11, -25,  30,
         5, -27, -17,  -6,  24, -27])

### Get ECOs

In [20]:
ecos = get_ecos()
ecos = pd.DataFrame(ecos, index=range(0, 2*len(ecos),2), columns=['ECO'])
ecos = ecos.reindex(range(0, len(ecos))).fillna(method='pad')

### Game features

In [49]:
import time

start = time.time()
games = get_games()
X, names = games_features(games, return_names=True)
game_features = pd.DataFrame(X.toarray(), columns=names)
print(time.time()-start)
game_features

15186.436979055405


,black_king_castle,black_queen_castle,can_claim_draw,end_B,end_K,end_N,end_P,end_Q,end_R,end_b,...,end_r,first_check_at,insufficient_material,promotion,queen_changed_at,queen_moved_at,total_checks,total_moves,white_king_castle,white_queen_castle
0,5,0,0,1,1,1,6,1,2,1,...,2,14,0,0,19,9,1,19,7,0
1,0,0,0,2,1,2,7,1,2,2,...,2,0,0,0,0,0,0,7,7,0
2,7,0,0,0,1,0,2,0,1,0,...,1,13,0,0,16,13,8,53,8,0
3,7,0,0,0,1,1,4,0,1,1,...,1,24,0,0,22,7,5,39,0,0
4,0,0,0,2,1,1,4,1,2,0,...,2,15,0,0,0,5,4,25,8,0
5,6,0,0,0,1,0,5,1,1,0,...,1,0,0,0,0,8,0,29,12,0
6,7,0,0,1,1,0,4,1,1,1,...,1,18,0,0,0,11,3,38,10,0
7,14,0,0,0,1,0,0,0,0,0,...,0,24,0,0,27,4,10,64,0,12
8,0,0,0,0,1,0,5,0,1,1,...,0,7,0,0,9,6,3,46,0,0
9,0,14,0,1,1,0,5,1,1,1,...,0,17,0,0,0,11,3,30,7,0


In [53]:
game_features.to_pickle('game_features.pickle')
game_features.to_csv('game_features.csv', index=False)